In [9]:
import os
import torch
from datasets import load_dataset
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import pandas as pd

In [106]:
df = pd.read_csv('Live_Usage_queries_with_mql_processed_rev3_curated.csv')

In [2]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/llama-2-7b-chat-hf"

# The instruction dataset to use
#dataset_name = ""

# Fine-tuned model name
new_model = "llama-2-7b-fine-tuned-peft-02"

In [3]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 16

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

In [4]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [5]:
################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "/output"

# Number of training epochs
num_train_epochs = 100

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True


# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = 10

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 10

In [6]:
################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [87]:
prompt_template = """### _LUMIN_ Question: {question} ### Context: {context} ### _LUMIN_ Answer: {answer}"""

In [88]:
context = "Converting above _LUMIN_ Question to LUMIN specific template _LUMIN_ Answer."

In [43]:
df.columns

Index(['query', 'template_query_fuzzy'], dtype='object')

In [89]:
def create_fine_tuning_dataset(row):
    question = row['query']
    answer = row['template_query_fuzzy']
    formated = prompt_template.format(question=question,context=context,
                                      answer=answer)
    data_point = str({'question': formated, "answer":answer})
    return data_point

In [45]:
df['fine_tuning_dataset']=df.apply(create_fine_tuning_dataset, axis=1)

In [46]:
df.drop(columns=['query','template_query_fuzzy'], inplace=True)

In [47]:
dataset = Dataset.from_pandas(df)

In [48]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [17]:
compute_dtype

torch.float16

In [18]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [19]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" 

In [21]:
# LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [49]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=200,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

In [50]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="fine_tuning_dataset",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/opt/conda/lib/python3.8/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/2543 [00:00<?, ? examples/s]

In [51]:
dataset[0]

{'fine_tuning_dataset': "{'question': '### _LUMIN_ Question:\\nwhy profit change in july 2022\\n\\n### Context:\\nAbove _LUMIN_ Question is converted into below _LUMIN_ Answer. This has to be strictly converted into LUMIN context only.\\n\\n### _LUMIN_ Answer:\\nWhy profit changed in july 2022\\n', 'answer': 'Why profit changed in july 2022'}"}

In [52]:
# Train model
trainer.train()

Step,Training Loss
10,2.211500
20,1.075800
30,0.398500
40,0.290000
50,0.420300
60,0.260100
70,0.132700
80,0.076100
90,0.140400
100,0.210400


TrainOutput(global_step=200, training_loss=0.328887548148632, metrics={'train_runtime': 1102.3523, 'train_samples_per_second': 0.726, 'train_steps_per_second': 0.181, 'total_flos': 1845780571324416.0, 'train_loss': 0.328887548148632, 'epoch': 0.31})

In [53]:
# Fine-tuned model name
new_model = "llama-2-7b-fine-tuned-peft-03"

In [54]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [55]:
from peft import PeftModel, PeftConfig

In [67]:
model_to_merge = PeftModel.from_pretrained(
    model,
    'llama-2-7b-fine-tuned-peft-02')

In [98]:
prompt_template = """### _LUMIN_ Question:
{question}

### Context:
{context}

"""

In [103]:
def predict_template_query(user_query):
    question = prompt_template.format(question=user_query, context=context)
    _input = str({'question': question})
    print(_input)
    _inputs = tokenizer.encode(_input, return_tensors="pt")
    outputs = model_to_merge.generate(input_ids=_inputs.to('cuda'), max_length= 200)
    output = tokenizer.decode(outputs[0])
    # print(output)
    # a = output.split("_LUMIN_ Answer",1)[1]
    #print(a)
    return output

In [35]:
inp = 'why sales decrease in 4th quarter 2023'
predict_template_query(inp)

/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:2415: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is /home/mosaic-ai/.cache/torch/kernels. This warning will appear only once per process. (Triggered internally at ../aten/src/ATen/native/cuda/jit_utils.cpp:1442.)
  next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)


'Why sales changed in quarter4 2023'

In [36]:
inp = "why production dropped for bikes in 1st quarter 2022"
predict_template_query(inp)

'Why production of bikes changed in quarter1 2022'

In [104]:
inp = 'why sales of texas increase in 1st quarter 2023'
a = predict_template_query(inp)

{'question': '### _LUMIN_ Question:\nwhy sales of texas increase in 1st quarter 2023\n\n### Context:\nConverting above _LUMIN_ Question to LUMIN specific template _LUMIN_ Answer.\n\n'}


In [105]:
a

"<s> {'question': '### _LUMIN_ Question:\\nwhy sales of texas increase in 1st quarter 2023\\n\\n### Context:\\nConverting above _LUMIN_ Question to LUMIN specific template _LUMIN_ Answer.\\n\\n'}\n nobody worked in 2023 \n\n### _LUMIN_ Question:\nWhy sales of texas changed in quarter1 2023\n\n### Context:\nConverting above _LUMIN_ Question to LUMIN specific template _LUMIN_ Answer\n\n### _LUMIN_ Answer:\nWhy sales of texas changed in quarter1 2023\n\nThis is the converted _LUMIN_ Question and Answer\n\nWhy sales of texas changed in quarter1 2023\n\nWhy sales of texas changed in quarter1 "

In [85]:
b=a.split('_LUMIN_ Answer')[1]
b.split('\n')[1]

'Why sales of texas changed in quarter1 2023'

In [62]:
inp = 'drivers of profit'
predict_template_query(inp)

<s> ### _LUMIN_ Question:
drivers of profit

### Context:
Above _LUMIN_ Question is converted into below _LUMIN_ Answer. This has to be strictly converted into LUMIN context only.

### _LUMIN_ Answer:
Drivers of profit

### Note:
This has to be strictly converted into LUMIN context only.

### _LUMIN_ Question:
What are the drivers of profit?

Please let me know if this is not converted correctly.</s>


''

In [ ]:
inp = "why sales dropped for bikes in 1st quarter 2022"
predict_template_query(inp).split('\n')[1]